In [ ]:
pip install patchify

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# example of preparing the horses and zebra dataset
from os import listdir
from numpy import asarray
from numpy import vstack
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img  
from numpy import savez_compressed
import numpy as np
#from patchify import patchify
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
drive.mount('/content/drive')
from patchify import patchify, unpatchify

In [ ]:
import keras
clas = keras.models.load_model("/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2. Motion_blur_dataset_Jan_23/Patches filtered/MB_text_blank_classifier.h5")


In [ ]:
#PATCH GENERATION

a = ['/content/drive/Shareddrives/IDP/Cycle GAN Implentation/1. Out_of_focus_Dataset_Jan 23/Out_of_focus_data_raw/OOF_Test_OG']


for l in range(len(a)):
  path = a[l] + '/'
  for filename in listdir(path):
    #try:  
    image = Image.open(path + filename)  # for example (3456, 5184, 3)
    image = np.asarray(image)
    patches = patchify(image, (256, 256, 3), step=256)
    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            patch = patches[i, j, 0]
            patch = Image.fromarray(patch)
            num = i * patches.shape[1] + j
            patch.save('/content/ptch1/' + f"{filename.split('.')[0]}patch{num}.jpg")
    # except:
    #   print(filename)

In [ ]:
#Code for saving all images - 2,5,17,20,24,26,28
len = 0
len2 = 0
for a in range(1,31):
  for b in [1,2]:
    if (((a in [5, 20, 24, 26, 28]) & (b == 1)) | ((a in [2,17,26]) & (b==2))):
      None
    else:
      img_name = 'S_Img_Android_D' + str(a) + '_L2_r35_a0_b0_Mb' + str(b) + '.jpg'
    #img_name = 'S_Img_Android_D' + str(a) + '_L2_r35_a0_b0' + '.jpg'
      image = Image.open('/content/drive/Shareddrives/IDP/VisionAPI/IMAGES/SmartdocQA/' + img_name)
      image = np.asarray(image)
      patches = patchify(image, (256, 256, 3), step=256) 
      for i in range(patches.shape[0]):
          for j in range(patches.shape[1]):
              patch = patches[i, j, 0]
              patch1 = patch.reshape(1,256,256,3)
              x = clas.predict(patch1, verbose = 0)[0][0]
              if  x == 0:  
                patch = Image.fromarray(patch)
                num = i * patches.shape[1] + j
                if a in range(1,11):
                  patch.save('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/1-10/' + f"{img_name.split('.')[0]}patch{num}.jpg")
                elif a in range(11,21):
                  patch.save('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/11-20/' + f"{img_name.split('.')[0]}patch{num}.jpg")
                else:
                  patch.save('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/21-30/' + f"{img_name.split('.')[0]}patch{num}.jpg")
                # if x == 0:
                #   patch.save('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/OG1/' + f"{img_name.split('.')[0]}patch{num}.jpg")
                len = len + 1
                # elif x < 0.4:
                #   patch.save('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/OG2/' + f"{img_name.split('.')[0]}patch{num}.jpg")
                #   len2 = len2 + 1


In [ ]:
#patches of orig images
for x in os.listdir('/content/drive/Shareddrives/IDP/VisionAPI/IMAGES/Random_sample_Clean_images for GAN/'):
  image = Image.open('/content/drive/Shareddrives/IDP/VisionAPI/IMAGES/Random_sample_Clean_images for GAN/' + x)
  image = np.asarray(image)
  print('IMAGE SHAPE', image.shape)
  patches = patchify(image, (256, 256, 3), step=256) 
  for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        patch = patches[i, j, 0]
        patch1 = patch.reshape(1,256,256,3)
        y = clas.predict(patch1, verbose = 0)[0][0]
        if y ==0:
          patch = Image.fromarray(patch)
          num = i * patches.shape[1] + j
          patch.save('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/OG2/' + f"{x.split('.')[0]}patch{num}.jpg")


In [ ]:
print(len(os.listdir('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/OG1/')) + len(os.listdir('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/OG2/')))
print(len(os.listdir('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/21-30/')) + len(os.listdir('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/1-10/')) + len(os.listdir('/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/11-20/')))

In [ ]:
!zip -r /content/Patches_OOF_blur_sample.zip /content/ptch1

In [ ]:
#npz file generation

path = '/content/drive/Shareddrives/IDP/Cycle GAN Implentation/0. All Blur dataset 30 Jan 23/All blur/'

# load all images in a directory into memory
size=(256,256)
OG = list();
Blr = list()
i=j=0;

# enumerate filenames in directory, assume all are images
for filename in listdir(path):
# load and resize the image
	try:
		pixels = load_img(path + filename, target_size=size)
		pixels = pixels.rotate(-90)
		# convert to numpy array
		pixels = img_to_array(pixels)
		#store
		if ((len(filename.split('_')) == 8)) : # & (i%4!=0)):
				i = i+1
				OG.append(pixels)
		elif ((len(filename.split('_')) == 9)):# & (j%4==0)):
				Blr.append(pixels)
				j = j+1
	except:
		print(filename)
	
print(i,j)
dataA = asarray(OG)
dataB = asarray(Blr)

print('Loaded data: ', dataA.shape, dataB.shape)
# save as compressed numpy array
filename = '/content/drive/Shareddrives/IDP/Cycle GAN Implentation/0. All Blur dataset 30 Jan 23/All blur/All Blur patches Jan 30.npz'
savez_compressed(filename, dataA, dataB)
print('Saved dataset: ', filename)
#print(OG.shape ,Blr.shape)

In [ ]:
#npz file generation

paths = ['/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/21-30/', '/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/1-10/','/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/11-20/']

# load all images in a directory into memory
size=(256,256)
OG = list();
Blr = list()
i=j=0;

# enumerate filenames in directory, assume all are images
for path in paths:
  for filename in listdir(path):
  # load and resize the image
    try:
      pixels = load_img(path + filename, target_size=size)
      pixels = pixels.rotate(-90)
      # convert to numpy array
      pixels = img_to_array(pixels)
      Blr.append(pixels)
      j = j+1
    except:
      print(filename)

paths = ['/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/OG1/', '/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/OG2/']
for path in paths:
  for filename in listdir(path):
  # load and resize the image
    try:
      pixels = load_img(path + filename, target_size=size)
      pixels = pixels.rotate(-90)
      # convert to numpy array
      pixels = img_to_array(pixels)
      OG.append(pixels)
      i = i+1
    except:
      print(filename)
	
print('OG & Blur count',i,j)
dataA = asarray(OG)
dataB = asarray(Blr)

print('Loaded data: ', dataA.shape, dataB.shape)
# save as compressed numpy array
filename = '/content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/MB_Blur_patches_all_files_May1.npz'
savez_compressed(filename, dataA, dataB)
print('Saved dataset: ', filename)
#print(OG.shape ,Blr.shape)


OG & Blur count 2329 2525
Loaded data:  (2329, 256, 256, 3) (2525, 256, 256, 3)
Saved dataset:  /content/drive/Shareddrives/IDP/Cycle GAN Implentation/2.b Motion_blur_dataset_all_images_Apr_23/MB_Blur_patches_all_files_May1.npz


In [ ]:
#npz file generation for test images

path = '/content/drive/Shareddrives/IDP/Cycle GAN Implentation/1. Out_of_focus_Dataset_Jan 23/PATCHES_FILTERED/Test_Patches_OOF_OG/'

# load all images in a directory into memory
size=(256,256)
Test = list();
i=0;

# enumerate filenames in directory, assume all are images
for filename in listdir(path):
  try:
    pixels = load_img(path + filename, target_size=size)
    pixels = pixels.rotate(-90)
		# convert to numpy array
    pixels = img_to_array(pixels)
    i = i+1
    Test.append(pixels)
  except:
    print(filename)
	
print(i)
dataA = asarray(Test)

print('Loaded data: ', dataA.shape)
# save as compressed numpy array
filename1 = '/content/OOF_OG.npz'
savez_compressed(filename1, dataA)
print('Saved dataset: ', filename1)
#print(OG.shape ,Blr.shape)